In [3]:
!pip install opencv-python
!pip install pytesseract


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import cv2
import pytesseract
from pytesseract import Output
import os


In [ ]:

# Ensure Tesseract OCR is properly configured
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCRtesseract.exe'

def extract_text_from_video(video_path, output_folder, interval=1):

    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_interval = interval * fps  # Number of frames to skip
    
    frame_count = 0
    results = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Process frame at the specified interval
        if frame_count % frame_interval == 0:
            timestamp = frame_count / fps  # Current time in seconds
            frame_filename = os.path.join(output_folder, f"frame_{frame_count}.jpg")
            
            # Save the frame for reference (optional)
            cv2.imwrite(frame_filename, frame)

            # Convert frame to grayscale for better OCR performance
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Optional: Apply preprocessing (e.g., thresholding)
            # gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

            # Perform OCR
            text = pytesseract.image_to_string(gray, lang='eng', config='--psm 6')
            if text.strip():  # Only consider non-empty text blocks
                results.append({'timestamp': timestamp, 'text': text.strip()})
                print(f"Extracted text at {timestamp:.2f}s: {text.strip()}")

        frame_count += 1

    cap.release()
    return results

# Path to the video file
video_path = "Video_text_recipe_opencv.mp4"

# Output folder for frames
output_folder = "extracted_frames"

# Extract text
text_blocks = extract_text_from_video(video_path, output_folder, interval=0.5)

# Save extracted text to a file
with open("extracted_text_blocks.txt", "w", encoding="utf-8") as f:
    for block in text_blocks:
        f.write(f"Time: {block['timestamp']:.2f}s\nText: {block['text']}\n\n")
